<a href="https://colab.research.google.com/github/Abhishekconfused/Models/blob/main/bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#my project starts here
import numpy as np #array
import pandas as pd #table
import matplotlib.pyplot as plt #plots
import re #substitution
import random #for pusedo random numbers
import nltk #natural language toolkit
from nltk.tokenize import TweetTokenizer
import string
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import collections
!pip install utils
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import utils

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [ ]:
!pip install lookup

In [ ]:
from lookup import lookup

In [ ]:

import pdb
from os import getcwd

In [ ]:
#function for preprocessing. a tweet contains various useless things which are not necessary for our model.I will remove them one by one.
def preprocess(tweet):
  # remove old style retweet text "RT"
  tweett = re.sub(r'^RT[\s]+', '', tweet)

  # remove hyperlinks
  tweettt = re.sub(r'https?:\/\/.*[\r\n]*', '', tweett)

  # remove hashtags
  # only removing the hash # sign from the word
  tweetttt = re.sub(r'#', '', tweettt)
  tweet2= re.sub(r',','',tweetttt)

  #lets tokenize that is break the tweet
  tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                                reduce_len=True)
  tweet3= tokenizer.tokenize(tweet2)
    #remove stopwords and panctuations
  import string
  stopwords=nltk.corpus.stopwords.words('english')
  tweet4=[]
  for words in tweet3:
    if words not in stopwords and words not in string.punctuation:
        tweet4.append(words)
  #lets stem
  stemmer=PorterStemmer()
  tweet5=[]
  for words in tweet4:
    stem=stemmer.stem(words)
    tweet5.append(stem)
   #lets lematize
  lemmatizer = WordNetLemmatizer()
  tweet6=[]

  for word in tweet5:
      lem_word = lemmatizer.lemmatize(word)  # stemming word
      tweet6.append(lem_word)  # append to the list


  return tweet6


In [ ]:
# def count_tweets(result,tweets,ys):
#   preprocess(tweets)
#   for y, tweet in zip(ys,tweets):
#     for word in preprocess(tweet):
#       pair=(word,y)
#       if pair in result:
#         result[pair]+=1
#       else:
#         result[pair]=1
#   return result
# def count_tweets(result, tweets, ys):
#     if not isinstance(tweets, str):
#         tweets = str(tweets)
#     preprocess(tweets)
#     for y, tweet in zip(ys, tweets):
#         for word in preprocess(tweet):
#             pair = (word, y)
#             if pair not in result:
#                 result[pair] = 0
#             result[pair] += 1
def build_freqs(tweet, ys):
    yslist = np.squeeze(ys).tolist()#the ys which is taken as input is squeezed so that it doesnot be a single element.

    freqs = {}
    for y, tweet in zip(yslist, tweet):
        for word in preprocess(tweet):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1

    return freqs




In [ ]:
p=twitter_samples.strings('positive_tweets.json')
n=twitter_samples.strings('negative_tweets.json')
t=p+n
y = np.append(np.ones((len(p), 1)), np.zeros((len(n), 1)), axis=0)

In [ ]:
#I will train the data using 90:10 ratio.
test_pos = p[4500:]
train_pos = p[:4500]
test_neg = n[4500:]
train_neg = n[:4500]
train_x = train_neg+train_pos
test_x = test_pos + test_neg
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [ ]:
freqs=build_freqs(train_x,train_y)


In [ ]:
def train_naive_bayes(freqs, train_x, train_y):
    loglikelihood = {}
    logprior = 0
    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    # calculate N_pos and N_neg
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] > 0:
            # Increment the number of positive words by the count for this (word, label) pair
            N_pos += freqs.get(pair, 1)
        # else, the label is negative
        else:
            # increment the number of negative words by the count for this (word,label) pair
            N_neg += freqs.get(pair, 1)
    # Calculate D, the number of documents
    D = len(train_y)
    # Calculate D_pos, the number of positive documents (*hint: use sum(<np_array>))
    D_pos = sum(train_y)
    # Calculate D_neg, the number of negative documents (*hint: compute using D and D_pos)
    D_neg = D-D_pos
    # Calculate logprior
    logprior = np.log(D_pos) - np.log(D_neg)
    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = freqs.get((word, 1),0)
        freq_neg = freqs.get((word, 0),0)
        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1)/(N_pos + V)
        p_w_neg = (freq_neg + 1)/(N_neg + V)
        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)
    return logprior, loglikelihood
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)


In [ ]:
def naive_bayes_predict(tweet, logprior, loglikelihood):

  # process the tweet to get a list of words
  word_l = preprocess(tweet)
  # initialize probability to zero
  p = 0
  # add the logprior
  p += logprior
  for word in word_l:
# check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]
  return p

In [ ]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):

    accuracy = 0  # return this properly
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0
# append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.absolute(y_hats - test_y))
# Accuracy is 1 minus the error
    accuracy = 1 - error
    return accuracy

0.5